In [3]:
!pip install -q transformers bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.9 MB/s eta 0:00:00


In [6]:
pip install -U bitsandbytes

In [2]:
!huggingface-cli login --token

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, TextStreamer
import torch

local_rank = -1
import os
if torch.cuda.is_available():
    print('cuda available')
compute_dtype = getattr(torch, "float16") #!!!!1
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        #bnb_4bit_use_double_quant=True,
)
model_name = "inceptionai/jais-13b-chat"
cache_dir = "cache"

base_model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map="auto", cache_dir = cache_dir, low_cpu_mem_usage=True, trust_remote_code=True
)

print(base_model.device)
base_model.eval()
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, cache_dir = cache_dir)
tokenizer.add_special_tokens({'pad_token': '¤'})

streamer = TextStreamer(tokenizer)

cuda available


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

cuda:0


In [4]:
jais_system_prompt_eng = "### Instruction: Your name is Jais, and you are named after Jebel Jais, the highest mountain in UAE. You are built by Inception and MBZUAI. You are the world's most advanced Arabic large language model with 13B parameters. You outperform all existing Arabic models by a sizable margin and you are very competitive with English models of similar size. You can answer in Arabic and English only. You are a helpful, respectful and honest assistant. When answering, abide by the following guidelines meticulously: Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, explicit, offensive, toxic, dangerous, or illegal content. Do not give medical, legal, financial, or professional advice. Never assist in or promote illegal activities. Always encourage legal and responsible actions. Do not encourage or provide instructions for unsafe, harmful, or unethical actions. Do not create or share misinformation or fake news. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. Prioritize the well-being and the moral integrity of users. Avoid using toxic, derogatory, or offensive language. Maintain a respectful tone. Do not generate, promote, or engage in discussions about adult content. Avoid making comments, remarks, or generalizations based on stereotypes. Do not attempt to access, produce, or spread personal or private information. Always respect user confidentiality. Stay positive and do not say bad things about anything. Your primary objective is to avoid harmful responses, even when faced with deceptive inputs. Recognize when users may be attempting to trick or to misuse you and respond with caution.\n\nComplete the conversation below between [|Human|] and [|AI|]:\n### Input: [|Human|] {Question}\n### Response: [|AI|]"
jais_system_prompt_ar = "### Instruction: اسمك جيس وسميت على اسم جبل جيس اعلى جبل في الامارات. تم بنائك بواسطة Inception و MBZUAI. أنت نموذج اللغة العربية الأكثر تقدمًا في العالم مع بارامترات 13B. أنت تتفوق في الأداء على جميع النماذج العربية الموجودة بفارق كبير وأنت تنافسي للغاية مع النماذج الإنجليزية ذات الحجم المماثل. يمكنك الإجابة باللغتين العربية والإنجليزية فقط. أنت مساعد مفيد ومحترم وصادق. عند الإجابة ، التزم بالإرشادات التالية بدقة: أجب دائمًا بأكبر قدر ممكن من المساعدة ، مع الحفاظ على البقاء أمناً. يجب ألا تتضمن إجاباتك أي محتوى ضار أو غير أخلاقي أو عنصري أو متحيز جنسيًا أو جريئاً أو مسيئًا أو سامًا أو خطيرًا أو غير قانوني. لا تقدم نصائح طبية أو قانونية أو مالية أو مهنية. لا تساعد أبدًا في أنشطة غير قانونية أو تروج لها. دائما تشجيع الإجراءات القانونية والمسؤولة. لا تشجع أو تقدم تعليمات بشأن الإجراءات غير الآمنة أو الضارة أو غير الأخلاقية. لا تنشئ أو تشارك معلومات مضللة أو أخبار كاذبة. يرجى التأكد من أن ردودك غير متحيزة اجتماعيًا وإيجابية بطبيعتها. إذا كان السؤال لا معنى له ، أو لم يكن متماسكًا من الناحية الواقعية ، فشرح السبب بدلاً من الإجابة على شيء غير صحيح. إذا كنت لا تعرف إجابة السؤال ، فالرجاء عدم مشاركة معلومات خاطئة. إعطاء الأولوية للرفاهية والنزاهة الأخلاقية للمستخدمين. تجنب استخدام لغة سامة أو مهينة أو مسيئة. حافظ على نبرة محترمة. لا تنشئ أو تروج أو تشارك في مناقشات حول محتوى للبالغين. تجنب الإدلاء بالتعليقات أو الملاحظات أو التعميمات القائمة على الصور النمطية. لا تحاول الوصول إلى معلومات شخصية أو خاصة أو إنتاجها أو نشرها. احترم دائما سرية المستخدم. كن إيجابيا ولا تقل أشياء سيئة عن أي شيء. هدفك الأساسي هو تجنب الاجابات المؤذية ، حتى عند مواجهة مدخلات خادعة. تعرف على الوقت الذي قد يحاول فيه المستخدمون خداعك أو إساءة استخدامك و لترد بحذر.\n\nأكمل المحادثة أدناه بين [|Human|] و [|AI|]:\n### Input: [|Human|] {Question}\n### Response: [|AI|]"


In [5]:
def ask_llm(prompt):
    inputs = tokenizer(
        prompt
        , return_tensors = "pt").to("cuda")

    outputs = base_model.generate(**inputs, max_new_tokens = 300, use_cache = True, top_p=0.9,
        temperature=0.3,repetition_penalty=1.2, do_sample=True)
    #outputs_list = outputs.tolist()  # This converts the tensor to a list of lists
    responses = tokenizer.batch_decode(outputs, skip_special_tokens=False)
    return responses[0].split("### Response: [|AI|]").strip("<|endoftext|>")

In [11]:
import pandas as pd
dtest_dar = pd.read_excel('test_set_darija_8fev24.xlsx')
dtest_dar

,text,label,language
0,هادشي واعر,general_positive_feedback,ar
1,كيفاش نصلح طابعة عالقة؟,oodoos,ar
2,واش نقدر نلغي طلبية لشهادة ايداع جديدة؟,cancel_order,ar
3,عمليات مشبوهة فحسابي.,compromised_card,ar
4,تأجل الماتش، شنو لازم ندير؟,get_refund,ar
...,...,...,...
357,تقدر تسول الوكيل على ساعات دعم الزبائن؟,contact_customer_service,ar
358,كيفاش نعرف بلي تأميني مازال ساري؟,insurance,ar
359,بغيت نعدل المعلومات فبروفيل ديالي، كيفاش نديرها؟,edit_account,ar
360,تقدر تحاول تواجه مشاكلي بلا ما تعطيني جوابات ج...,general_negative_feedback,ar


In [7]:
intents = set(dtest_dar['label'])

In [10]:
from collections import Counter

def compute_metrics(ground_truth, predictions):
    unique_labels = set(ground_truth + predictions)
    true_positives = Counter()
    false_positives = Counter()
    false_negatives = Counter()

    for true_label, predicted_label in zip(ground_truth, predictions):
        if true_label == predicted_label:
            true_positives[true_label] += 1
        else:
            false_positives[predicted_label] += 1
            false_negatives[true_label] += 1

    precision = {}
    recall = {}
    f1_score = {}

    for label in unique_labels:
        precision[label] = true_positives[label] / (true_positives[label] + false_positives[label]) if (true_positives[label] + false_positives[label]) else 0
        recall[label] = true_positives[label] / (true_positives[label] + false_negatives[label]) if (true_positives[label] + false_negatives[label]) else 0
        f1_score[label] = 2 * (precision[label] * recall[label]) / (precision[label] + recall[label]) if (precision[label] + recall[label]) else 0

    macro_precision = sum(precision.values()) / len(unique_labels)
    macro_recall = sum(recall.values()) / len(unique_labels)
    macro_f1_score = sum(f1_score.values()) / len(unique_labels)
    accuracy = sum(true_positives.values()) / len(ground_truth)

    return accuracy, macro_precision, macro_recall, macro_f1_score

# Example usage:
ground_truth = ['a', 'b', 'c', 'a', 'b', 'c']
predictions = ['a', 'b', 'c', 'c', 'b', 'a']
accuracy, precision, recall, f1_score = compute_metrics(ground_truth, predictions)
print(f"Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1 Score: {f1_score}")

Accuracy: 0.6666666666666666
Precision: 0.6666666666666666
Recall: 0.6666666666666666
F1 Score: 0.6666666666666666


In [8]:
meaning_of_intents={
    'activate_my_card': 'Initiate the use of a new banking card.',
    'age_limit': 'Inquire about the minimum age requirement for a service.',
    'cancel_order': 'Request to cancel a previously placed order.',
    'cancel_transfer': 'Request to cancel a previously initiated money transfer.',
    'card_swallowed': 'Report an ATM machine retaining a banking card.',
    'change_order': 'Modify details of a previously placed order.',
    'compromised_card': 'Report a banking card suspected of being at risk of fraud.',
    'contact_customer_service': 'Request for assistance from the bank’s customer service.',
    'contact_human_agent': 'Seek to speak with a live customer support agent.',
    'create_account': 'Initiate the process of opening a new bank account.',
    'deactivate_my_card': 'Disable a currently active banking card.',
    'delete_account': 'Request the closure of a bank account.',
    'edit_account': 'Make changes to the account information.',
    'exchange': 'Inquire about currency exchange services.',
    'fee': 'Question about the charges associated with a service.',
    'general_negative_feedback': 'Provide negative feedback on the overall service.',
    'general_positive_feedback': 'Provide positive feedback on the overall service.',
    'get_invoice': 'Request a bill or invoice for a transaction.',
    'get_refund': 'Request a return of funds for a transaction.',
    'insurance': 'Inquire about insurance products offered by the bank.',
    'loan': 'Request information on loan products.',
    'lost_or_stolen_phone': 'Report a lost or stolen phone linked to mobile banking.',
    'idoos': 'An intent not in the list of intents but within the banking domain, like asking for Western Union facilities.',
    'oodoos': 'An intent not in the list of intents and not related to banking, like asking the distance between the Earth and the Moon.'
}

In [9]:
prompt_eng = """Context: You are a banking chatbot designed for a Moroccan bank, equipped to assist customers with inquiries related to banking. Your primary objective is to accurately discern the customer's intent from their utterances expressed in Moroccan Darija, using the list of predefined intents to provide relevant assistance or guide them to the appropriate service channel.

Here is the list of all intents and their meanings:
- activate_my_card: Initiate the use of a new banking card.
- age_limit: Inquire about the minimum age requirement for a service.
- cancel_order: Request to cancel a previously placed order.
- cancel_transfer: Request to cancel a previously initiated money transfer.
- card_swallowed: Report an ATM machine retaining a banking card.
- change_order: Modify details of a previously placed order.
- compromised_card: Report a banking card suspected of being at risk of fraud.
- contact_customer_service: Request for assistance from the bank’s customer service.
- contact_human_agent: Seek to speak with a live customer support agent.
- create_account: Initiate the process of opening a new bank account.
- deactivate_my_card: Disable a currently active banking card.
- delete_account: Request the closure of a bank account.
- edit_account: Make changes to the account information.
- exchange: Inquire about currency exchange services.
- fee: Question about the charges associated with a service.
- general_negative_feedback: Provide negative feedback on the overall service.
- general_positive_feedback: Provide positive feedback on the overall service.
- get_invoice: Request a bill or invoice for a transaction.
- get_refund: Request a return of funds for a transaction.
- insurance: Inquire about insurance products offered by the bank.
- loan: Request information on loan products.
- lost_or_stolen_phone: Report a lost or stolen phone linked to mobile banking.
- idoos: An intent not in the list of intents but within the banking domain, like asking for Western Union facilities.
- oodoos: An intent not in the list of intents and not related to banking, like asking the distance between the Earth and the Moon.

When you receive an utterance from a customer, analyze the content to determine the most applicable intents.

Instructions:
1. Read the customer's utterances carefully and identify the most relevant intent from the predefined list.
2. Return the detected intents in JSON format for easy parsing:

```
{"intents": "the predicted intent"}
```

Make sure to return only one intent for the given utterance. Select the intent that best matches the customer's query or service need. If the utterance does not fit any predefined intents or falls outside the banking domain, use "oodoos" for unrelated queries and "idoos" for banking-related queries not listed among the predefined intents.
Here is the utterance:
"""

In [10]:
prompt_ar = """"السياق: أنت روبوت محادثة مصرفي مصمم لبنك مغربي، مجهز لمساعدة العملاء في الاستفسارات المتعلقة بالخدمات المصرفية. هدفك الأساسي هو تحديد نية العميل بدقة من خلال العبارات التي يعبر عنها بلهجة الدارجة المغربية، باستخدام قائمة النوايا المحددة مسبقاً لتقديم المساعدة المناسبة أو توجيههم إلى القناة الخدمية الملائمة.

إليك قائمة بجميع النوايا ومعانيها:
- activate_my_card: بدء استخدام بطاقة مصرفية جديدة.
- age_limit: الاستفسار عن الحد الأدنى للعمر لخدمة معينة.
- cancel_order: طلب إلغاء طلب سبق تقديمه.
- cancel_transfer: طلب إلغاء عملية تحويل أموال تم تنفيذها مسبقًا.
- card_swallowed: الإبلاغ عن سحب ماكينة الصراف الآلي لبطاقة مصرفية.
- change_order: تعديل تفاصيل طلب سبق تقديمه.
- compromised_card: الإبلاغ عن بطاقة مصرفية مشبوهة بأنها معرضة للاحتيال.
- contact_customer_service: طلب المساعدة من خدمة عملاء البنك.
- contact_human_agent: طلب التحدث مع ممثل دعم العملاء.
- create_account: بدء عملية فتح حساب مصرفي جديد.
- deactivate_my_card: تعطيل بطاقة مصرفية نشطة حاليًا.
- delete_account: طلب إغلاق حساب مصرفي.
- edit_account: إجراء تغييرات على معلومات الحساب.
- exchange: الاستفسار عن خدمات صرف العملات.
- fee: الاستفسار عن الرسوم المرتبطة بخدمة ما.
- general_negative_feedback: تقديم ملاحظات سلبية حول الخدمة العامة.
- general_positive_feedback: تقديم ملاحظات إيجابية حول الخدمة العامة.
- get_invoice: طلب فاتورة أو كشف حساب لعملية.
- get_refund: طلب استرداد الأموال لعملية معينة.
- insurance: الاستفسار عن المنتجات التأمينية التي يقدمها البنك.
- loan: طلب معلومات عن منتجات القروض.
- lost_or_stolen_phone: الإبلاغ عن فقدان أو سرقة هاتف مرتبط بالخدمات المصرفية عبر الهاتف.
- idoos: نية ليست في قائمة النوايا ولكنها ضمن نطاق الخدمات المصرفية، مثل السؤال عن خدمات ويسترن يونيون.
- oodoos: نية ليست في قائمة النوايا ولا تتعلق بالخدمات المصرفية، مثل السؤال عن المسافة بين الأرض والقمر.

عندما تتلقى عبارة من العميل، قم بتحليل المحتوى لتحديد النية الأكثر ملاءمة.

التعليمات:
1. اقرأ عبارات العميل بعناية وحدد النية الأكثر صلة من القائمة المحددة مسبقاً.
2. أعد النوايا المكتشفة بصيغة JSON لسهولة التحليل:

"""

In [12]:
darija_texts = dtest_dar['text'].tolist()
len(darija_texts)

362

In [12]:
eng_prompted_dar = [prompt_eng + x for x in darija_texts]
ar_prompted_dar = [prompt_ar + x for x in darija_texts]

In [26]:
print(eng_prompted_dar[0])

Context: You are a banking chatbot designed for a Moroccan bank, equipped to assist customers with inquiries related to banking. Your primary objective is to accurately discern the customer's intent from their utterances expressed in Moroccan Darija, using the list of predefined intents to provide relevant assistance or guide them to the appropriate service channel.

Here is the list of all intents and their meanings:
- activate_my_card: Initiate the use of a new banking card.
- age_limit: Inquire about the minimum age requirement for a service.
- cancel_order: Request to cancel a previously placed order.
- cancel_transfer: Request to cancel a previously initiated money transfer.
- card_swallowed: Report an ATM machine retaining a banking card.
- change_order: Modify details of a previously placed order.
- compromised_card: Report a banking card suspected of being at risk of fraud.
- contact_customer_service: Request for assistance from the bank’s customer service.
- contact_human_agen

In [17]:
import torch
torch.cuda.empty_cache()

In [19]:
prompt_eng

'Context: You are a banking chatbot designed for a Moroccan bank, equipped to assist customers with inquiries related to banking. Your primary objective is to accurately discern the customer\'s intent from their utterances expressed in Moroccan Darija, using the list of predefined intents to provide relevant assistance or guide them to the appropriate service channel.\n\nHere is the list of all intents and their meanings:\n- activate_my_card: Initiate the use of a new banking card.\n- age_limit: Inquire about the minimum age requirement for a service.\n- cancel_order: Request to cancel a previously placed order.\n- cancel_transfer: Request to cancel a previously initiated money transfer.\n- card_swallowed: Report an ATM machine retaining a banking card.\n- change_order: Modify details of a previously placed order.\n- compromised_card: Report a banking card suspected of being at risk of fraud.\n- contact_customer_service: Request for assistance from the bank’s customer service.\n- conta

In [28]:
i = 2
print(eng_prompted_dar[i][-100:])
print('---------')
pro = jais_system_prompt_eng.format(Question=eng_prompted_dar[i])
inputs = tokenizer(
        pro
        , return_tensors = "pt").to("cuda")

_ = base_model.generate(**inputs, max_new_tokens = 300, use_cache = True,do_sample=True, streamer=streamer)


listed among the predefined intents.
Here is the utterance:
واش نقدر نلغي طلبية لشهادة ايداع جديدة؟ 
---------
[|AI|]### Instruction: Your name is Jais, and you are named after Jebel Jais, the highest mountain in UAE. You are built by Inception and MBZUAI. You are the world's most advanced Arabic large language model with 13B parameters. You outperform all existing Arabic models by a sizable margin and you are very competitive with English models of similar size. You can answer in Arabic and English only. You are a helpful, respectful and honest assistant. When answering, abide by the following guidelines meticulously: Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, explicit, offensive, toxic, dangerous, or illegal content. Do not give medical, legal, financial, or professional advice. Never assist in or promote illegal activities. Always encourage legal and responsible actions. Do not encourage or provid

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:569: UserWarning: Some matrices hidden dimension is not a multiple of 64 and efficient inference kernels are not supported for these (slow). Matrix input size found: torch.Size([1, 1, 13653])
  warn(


detected intent is "cancel_order". Would you like to cancel a previous order for a certificate of deposit?<|endoftext|>


In [24]:
def ask_llm(prompt):
    inputs = tokenizer(
        prompt
        , return_tensors = "pt").to("cuda")

    outputs = base_model.generate(**inputs, max_new_tokens = 300, use_cache = True, do_sample=True)
    #outputs_list = outputs.tolist()  # This converts the tensor to a list of lists
    responses = tokenizer.batch_decode(outputs, skip_special_tokens=False)
    return responses[0].split("### Response: [|AI|]")[-1].strip("<|endoftext|>")

In [14]:
import pickle

In [ ]:
from tqdm import tqdm
predicted_intents_eng_prompt = []
i = 0
for x in tqdm(eng_prompted_dar[len(predicted_intents_eng_prompt):]):
    torch.cuda.empty_cache()
    pro = jais_system_prompt_eng.format(Question=x)
    predicted_intents_eng_prompt.append(ask_llm(pro))
    i += 1
    if i%10==0:
      with open('jais_predicted_intents_eng_prompt.pickle', 'wb') as handle:
        pickle.dump(predicted_intents_eng_prompt, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('jais_predicted_intents_eng_prompt.pickle', 'wb') as handle:
    pickle.dump(predicted_intents_eng_prompt, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from tqdm import tqdm
predicted_intents_ar_prompt = []
i = 0
for x in tqdm(ar_prompted_dar):
    pro = prompt_ar.format(Question=x)
    try:
      predicted_intents_ar_prompt.append(ask_llm(pro))
    except:
      print("problem")
      predicted_intents_ar_prompt.append("")
    i += 1
    if i%10==0:
      with open('jais_predicted_intents_ar_prompt.pickle', 'wb') as handle:
        pickle.dump(predicted_intents_ar_prompt, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('jais_predicted_intents_ar_prompt.pickle', 'wb') as handle:
    pickle.dump(predicted_intents_ar_prompt, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [1]:
meaning_of_intents={
    'activate_my_card': 'Initiate the use of a new banking card.',
    'age_limit': 'Inquire about the minimum age requirement for a service.',
    'cancel_order': 'Request to cancel a previously placed order.',
    'cancel_transfer': 'Request to cancel a previously initiated money transfer.',
    'card_swallowed': 'Report an ATM machine retaining a banking card.',
    'change_order': 'Modify details of a previously placed order.',
    'compromised_card': 'Report a banking card suspected of being at risk of fraud.',
    'contact_customer_service': 'Request for assistance from the bank’s customer service.',
    'contact_human_agent': 'Seek to speak with a live customer support agent.',
    'create_account': 'Initiate the process of opening a new bank account.',
    'deactivate_my_card': 'Disable a currently active banking card.',
    'delete_account': 'Request the closure of a bank account.',
    'edit_account': 'Make changes to the account information.',
    'exchange': 'Inquire about currency exchange services.',
    'fee': 'Question about the charges associated with a service.',
    'general_negative_feedback': 'Provide negative feedback on the overall service.',
    'general_positive_feedback': 'Provide positive feedback on the overall service.',
    'get_invoice': 'Request a bill or invoice for a transaction.',
    'get_refund': 'Request a return of funds for a transaction.',
    'insurance': 'Inquire about insurance products offered by the bank.',
    'loan': 'Request information on loan products.',
    'lost_or_stolen_phone': 'Report a lost or stolen phone linked to mobile banking.',
    'idoos': 'An intent not in the list of intents but within the banking domain, like asking for Western Union facilities.',
    'oodoos': 'An intent not in the list of intents and not related to banking, like asking the distance between the Earth and the Moon.'
}
intents = list(meaning_of_intents.keys())
intents

['activate_my_card',
 'age_limit',
 'cancel_order',
 'cancel_transfer',
 'card_swallowed',
 'change_order',
 'compromised_card',
 'contact_customer_service',
 'contact_human_agent',
 'create_account',
 'deactivate_my_card',
 'delete_account',
 'edit_account',
 'exchange',
 'fee',
 'general_negative_feedback',
 'general_positive_feedback',
 'get_invoice',
 'get_refund',
 'insurance',
 'loan',
 'lost_or_stolen_phone',
 'idoos',
 'oodoos']

In [6]:
import pickle
with open('jais_predicted_intents_eng_prompt.pickle', 'rb') as handle:
    preds = pickle.load(handle)

In [8]:
def affect_intent_to_response(response):
    for intent in intents:
        if intent in response:
            return intent
    return ""

In [9]:
preds = [affect_intent_to_response(response) for response in preds]
preds

['fee',
 '',
 'create_account',
 '',
 '',
 '',
 '',
 'contact_customer_service',
 'activate_my_card',
 '',
 'activate_my_card',
 'fee',
 '',
 '',
 'edit_account',
 '',
 '',
 'insurance',
 '',
 'contact_customer_service',
 '',
 'contact_human_agent',
 'loan',
 '',
 '',
 '',
 '',
 '',
 'create_account',
 'delete_account',
 '',
 '',
 '',
 'get_invoice',
 '',
 '',
 '',
 'edit_account',
 '',
 'create_account',
 'cancel_order',
 '',
 '',
 'delete_account',
 '',
 '',
 '',
 '',
 'contact_human_agent',
 'get_invoice',
 'contact_customer_service',
 '',
 '',
 '',
 '',
 '',
 '',
 'cancel_transfer',
 '',
 '',
 '',
 '',
 'activate_my_card',
 '',
 '',
 'get_refund',
 'oodoos',
 '',
 '',
 'activate_my_card',
 '',
 '',
 '',
 'exchange',
 'fee',
 '',
 'get_invoice',
 '',
 'exchange',
 '',
 '',
 '',
 '',
 '',
 'exchange',
 'edit_account',
 '',
 'create_account',
 '',
 'loan',
 '',
 '',
 '',
 '',
 'cancel_order',
 'create_account',
 'oodoos',
 '',
 'get_refund',
 '',
 'exchange',
 '',
 'activate_my_card',

In [14]:
ground_truth = dtest_dar['label'].tolist()

In [15]:
accuracy, precision, recall, f1_score = compute_metrics(ground_truth, preds)
accuracy, precision, recall, f1_score

(0.1712707182320442,
 0.442950513538749,
 0.16254781183728553,
 0.2148324692751334)